In [58]:
import bs4
import requests
from elasticsearch import Elasticsearch
from elasticsearch_dsl import Search, Q
es = Elasticsearch([{'host': 'localhost', 'port': 9200}])

In [9]:
def getTrans(link):
    page = requests.get(link)
    soup = bs4.BeautifulSoup(page.text)
    return soup.find('div',{'id':'transcript'}).findAll(text = True)
    

In [10]:
def findnth(haystack, needle, n):
    parts= haystack.split(needle, n+1)
    if len(parts)<=n+1:
        return -1
    return len(haystack)-len(parts[-1])-len(needle)

In [23]:
def createJSON (link, text):
    lnk = 'http://millercenter.org' + link
    pres_id = link[findnth(link,'/',1)+1 : findnth(link,'/',2)]
    speech_id = link[findnth(link,'/',3)+1 : len(link)]
    ps = {}
    ps['transcript'] = getTrans(lnk)
    ps['link'] = lnk
    ps['title'] = text[0:text.find("(")]
    ps['pres_id'] = pres_id
    ps['es_id'] = pres_id +'_'+ speech_id
    ps['date'] = text[text.find("(")+1:text.find(")")]
    return ps


In [24]:
page = requests.get('http://millercenter.org/president/speeches')
soup = bs4.BeautifulSoup(page.text)
links = soup.select('div.title a[href^=/president]')
#print links
for link in links:
    ps = createJSON(link.attrs['href'],link.text)
    es.index(index='president', doc_type='speeches', id=ps['es_id'], body = ps) 

In [28]:
#es.get(index='president', doc_type='speeches', id='clinton_speech-4597')

In [29]:
es.count(index='president', doc_type='speeches')

{u'_shards': {u'failed': 0, u'successful': 5, u'total': 5}, u'count': 894}

## The below does not user the dls approach 

In [62]:
agg_json = '{"size":0,"aggregations": {"president":{"terms":{"field": "pres_id"}}}}'
es.search(index='president', doc_type='speeches', body = agg_json)
#"size":0,
#print response
#for item in response:
#    print item
    
#for tag in response['aggregations']['president']['buckets']:
#    print(tag['key'], tag['doc_count']['value'])

{u'_shards': {u'failed': 0, u'successful': 5, u'total': 5},
 u'aggregations': {u'president': {u'buckets': [{u'doc_count': 71,
     u'key': u'lbjohnson'},
    {u'doc_count': 59, u'key': u'reagan'},
    {u'doc_count': 49, u'key': u'fdroosevelt'},
    {u'doc_count': 45, u'key': u'kennedy'},
    {u'doc_count': 39, u'key': u'clinton'},
    {u'doc_count': 32, u'key': u'grant'},
    {u'doc_count': 32, u'key': u'wilson'},
    {u'doc_count': 31, u'key': u'johnson'},
    {u'doc_count': 30, u'key': u'cleveland'},
    {u'doc_count': 29, u'key': u'hoover'}],
   u'doc_count_error_upper_bound': 0,
   u'sum_other_doc_count': 477}},
 u'hits': {u'hits': [], u'max_score': 0.0, u'total': 894},
 u'timed_out': False,
 u'took': 1}

In [66]:
s = Search(using=es, index="president",doc_type="speeches")
    
s.aggs.bucket('president', 'terms', field='pres_id') 

response = s.execute()

print response
#for hit in response['hits']['hits']:
#    print(hit['_score'], hit['_source']['pres_id'])

<Response: [<Result(president/speeches/clinton_speech-4597): {u'title': u'Remarks at the Democratic National Convention '...}>, <Result(president/speeches/clinton_speech-5494): {u'title': u'State of the Union Address ', u'pres_id': u'cli...}>, <Result(president/speeches/clinton_speech-4566): {u'title': u'Address on Somalia ', u'pres_id': u'clinton', u...}>, <Result(president/speeches/clinton_speech-3436): {u'title': u'Remarks to the Convocation of the Church of God...}>, <Result(president/speeches/gwbush_speech-4542): {u'title': u'State of the Union Address ', u'pres_id': u'gwb...}>, <Result(president/speeches/clinton_speech-3929): {u'title': u'Address on Bosnia ', u'pres_id': u'clinton', u'...}>, <Result(president/speeches/gwbush_speech-3645): {u'title': u'First Inaugural Address ', u'pres_id': u'gwbush...}>, <Result(president/speeches/gwbush_speech-3986): {u'title': u'Remarks on Signing the Economic Growth and Tax ...}>, <Result(president/speeches/clinton_speech-3443): {u'title': u'S